In [ ]:
%pip install transformers
%pip install datasets
%pip install sentencepiece
%pip install rouge_score
%pip install nbformat
%pip install plotly
%pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 18.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.4/269.4 kB 26.9 MB/s eta 0:00:0

In [ ]:
# funcao para fazer um print para mostra o texto em diferentes linhas
import textwrap
def print_lines(texto,w=50):
    lineas = textwrap.wrap(texto, width=w)
    for linea in lineas:
        print(linea)

In [ ]:
from datasets import load_dataset,load_metric
# Load dataset from huggingface
dataset = load_dataset("exams", "crosslingual_with_para_pt")
dataset

Generating train split:   0%|          | 0/740 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/184 [00:00<?, ? examples/s]

Dataset exams downloaded and prepared to /root/.cache/huggingface/datasets/exams/crosslingual_with_para_pt/1.0.0/4330a7899d757352941782397b09a4603093ef04e23873967042f5cee03cadd8. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'answerKey', 'info'],
        num_rows: 740
    })
    validation: Dataset({
        features: ['id', 'question', 'answerKey', 'info'],
        num_rows: 184
    })
})

In [ ]:
# Manipulate data set to build inputs
questions = [x['stem'] for x in dataset['train']['question']]
subjects = [x['subject'] for x in dataset['train']['info']]
choices = [x['choices'] for x in dataset['train']['question']]
correto = list()
for answerKey in dataset['train']['answerKey']:
    correto.append('Sim' if answerKey=='A' else 'Não')
    correto.append('Sim' if answerKey=='B' else 'Não')
    correto.append('Sim' if answerKey=='C' else 'Não')
    correto.append('Sim' if answerKey=='D' else 'Não')
subjects = [x['subject'] for x in dataset['train']['info']]
inputs = ["Tópico: {} Questão: {} Context: {} Correto: {}".format(subjects[i//4], questions[i//4], choices[i//4]['para'][i%4], correto[i]) for i in range(4*len(questions))]

In [ ]:
# Manipulate dataset to build expected outputs
choices = [x['choices'] for x in dataset['train']['question']]
outputs = list()
for choice in choices:
    for i in range(4):
        outputs.append(choice['text'][i])

In [ ]:
prefix = "question: "
max_input_length = 256
max_target_length = 32

In [ ]:
def preprocess_examples(examples):
    """
    example: batch conjunto de exemplos
    Essa funcão é uma das mais importante já que formata nosso texto de entrada tokenizando o texto
    para esse finetunning usamos o dataset de um jeito diferente onde a entrada para o modelo sera uma string com tópico e questão
    e nossa saida esperada(outputs) serão alternativas.
    """
    questions = [x['stem'] for x in dataset['train']['question']]
    subjects = [x['subject'] for x in dataset['train']['info']]
    choices = [x['choices'] for x in dataset['train']['question']]
    correto = list()
    for answerKey in dataset['train']['answerKey']:
        correto.append('Sim' if answerKey=='A' else 'Não')
        correto.append('Sim' if answerKey=='B' else 'Não')
        correto.append('Sim' if answerKey=='C' else 'Não')
        correto.append('Sim' if answerKey=='D' else 'Não')
    subjects = [x['subject'] for x in dataset['train']['info']]
    inputs = ["Tópico: {} Questão: {} Context: {} Correto: {}".format(subjects[i//4], questions[i//4], choices[i//4]['para'][i%4], correto[i]) for i in range(4*len(questions))]
    outputs = list()
    for choice in choices:
        for i in range(4):
            outputs.append(choice['text'][i])
    # inputs e outputs sao texto agora eles vai ser tokenizados lembrar que model_inputs tem diferentes campos ver tutorial 1
    model_inputs = tokenizer(inputs, max_length=max_input_length, padding="max_length", truncation=True)
    # o label ficara só com os tokens_id
    labels = tokenizer(outputs, max_length=max_target_length, padding="max_length", truncation=True).input_ids
    labels_with_ignore_index = []
    for labels_example in labels:
        # devido ao processo de padding é importante não considerar esses ids
        labels_example = [label if label != 0 else -100 for label in labels_example]
        labels_with_ignore_index.append(labels_example)
    model_inputs["labels"] = labels_with_ignore_index
    return model_inputs

In [ ]:
from torch.utils.data import DataLoader
def create_dataloaders(train_batch_size=8, eval_batch_size=32):
    train_dataloader = DataLoader(encoded_train_ds, shuffle=True, batch_size=train_batch_size)
    val_dataloader = DataLoader(encoded_val_ds, shuffle=False, batch_size=eval_batch_size)

    return train_dataloader, val_dataloader

In [ ]:
from transformers import T5Tokenizer
# modelo base para o finetunning
checkpoint = 'unicamp-dl/ptt5-base-portuguese-vocab'
# inicializa o tokenizer
tokenizer = T5Tokenizer.from_pretrained(checkpoint,model_max_length=max_input_length)

In [ ]:
# separa o dataset
train,val = dataset['train'],dataset['validation']

In [ ]:
encoded_train_ds = train.map(preprocess_examples, batched=True, remove_columns=train.column_names)
encoded_val_ds = val.map(preprocess_examples, batched=True, remove_columns=val.column_names)

Map:   0%|          | 0/740 [00:00<?, ? examples/s]

Map:   0%|          | 0/184 [00:00<?, ? examples/s]

In [ ]:
encoded_train_ds.set_format(type="torch")
encoded_val_ds.set_format(type="torch")

In [ ]:
from transformers import EarlyStoppingCallback
from transformers import T5Model, T5ForConditionalGeneration

# PyTorch
model_pt = T5ForConditionalGeneration.from_pretrained(checkpoint)
# early stopping e uma tecnica onde o finetuning para quando ele nao melhorar depois de 3 vezes consecutivas
early = EarlyStoppingCallback(early_stopping_patience=3)

In [ ]:
import nltk
# essa biblioteca e importante para a funcao compute metrics
nltk.download('punkt')
metric = load_metric("rouge")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
<ipython-input-16-d6866b6a5e99>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


In [ ]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    # tem a predicoes e os labels(o que é esperado)
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # calculo das metricas
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pt)

In [ ]:
from transformers import Seq2SeqTrainingArguments

batch_size = 8
num_train_epochs = 30
# Show the training loss with every epoch
logging_steps = len(encoded_train_ds) // batch_size
# formatacao do modelo mudar se precisar para um padrao
model_name = "question2answer"
args = Seq2SeqTrainingArguments(
    output_dir=f"{model_name}",
    evaluation_strategy="epoch",
    learning_rate=5.6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    # armazena só um modelo no colab é imporante não usar muita memoria
    save_total_limit=1,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
    logging_steps=logging_steps,
    # garante que o modelo armazene o melhor modelo
    load_best_model_at_end=True,
    save_strategy="epoch",
    fp16=True
)

In [ ]:
from transformers import Seq2SeqTrainer
# Criamos o Objeto trainer que vai receber os argumentos anterios juntos com as entradas numericas
trainer = Seq2SeqTrainer(
    model_pt,
    args,
    train_dataset=encoded_train_ds,
    eval_dataset=encoded_val_ds,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[early]

)

In [ ]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,3.932900,2.711499,25.127400,10.232000,23.099100,23.104200,10.309100
2,2.817700,2.193566,30.268700,14.129300,27.912600,27.916100,9.987200
3,2.430200,1.834729,35.928900,19.339900,33.385200,33.394200,10.716900
4,2.152000,1.562052,40.548400,23.843100,38.095900,38.081700,9.862800
5,1.924800,1.354360,44.893300,28.187600,42.362800,42.368000,10.676700
6,1.740100,1.180266,47.466400,31.420200,45.138000,45.175900,10.597000
7,1.597200,1.031676,51.210800,35.652000,48.815200,48.886600,10.767900
8,1.450800,0.908462,54.112300,39.052300,51.866100,51.880000,10.601000
9,1.336400,0.813082,56.897400,42.270000,54.665400,54.665500,10.687200
10,1.243200,0.727746,59.414400,45.147300,57.186200,57.242100,10.936500


TrainOutput(global_step=11100, training_loss=1.1945357836474169, metrics={'train_runtime': 10781.6783, 'train_samples_per_second': 8.236, 'train_steps_per_second': 1.03, 'total_flos': 2.7037730340864e+16, 'train_loss': 1.1945357836474169, 'epoch': 30.0})

In [ ]:
model_pt.to("cpu")
model_pt.save_pretrained("temporal"+"_best/")

In [ ]:
val

Dataset({
    features: ['id', 'question', 'answerKey', 'info'],
    num_rows: 184
})

In [ ]:
val['question'][100]['choices']['para'][0]

'Banco Europeu de Investimento\n\nO Banco Europeu de Investimento (BEI), é instrumento financeiro da União Europeia. O Banco Mundial e o Banco Europeu de Investimento, por exemplo, só participam do financiamento de projetos no setor do carvão em casos excepcionais. Foi Vice-Presidente do Banco Europeu de Investimento entre 2007 e 2010. Desempenha ainda funções de administrador não executivo na Nutrinveste e no Banco Finantia, e de membro do Comité de Investimento da PVCI (Portugal Venture Capital Iniciative), um fundo de fundos de investimento ligado ao Banco Europeu de Investimento. A cidade do Luxemburgo contém várias instituições da União Europeia, incluindo o Tribunal de Justiça Europeu, o Tribunal de Contas e o Banco Europeu de Investimento. A cidade do Luxemburgo contém várias instituições da União Europeia, incluindo o Tribunal de Justiça Europeu, o Tribunal de Contas e o Banco Europeu de Investimento. A cidade do Luxemburgo contém várias instituições da União Europeia, incluind

In [ ]:
val['question'][100]['stem']

'No processo de aprovação do orçamento comunitário intervém o'

In [ ]:
val['question'][100]['choices']['text'][0]

'Banco Europeu de Investimento'

In [ ]:
query_correct = ["Tópico: {} Questão: {} Context: {} Correto: Sim".format(val['info'][100]['subject'], val['question'][100]['stem'], val['question'][100]['choices']['para'][0])]
query_wrong = ["Tópico: {} Questão: {} Context: {} Correto: Não".format(val['info'][100]['subject'], val['question'][100]['stem'], val['question'][100]['choices']['para'][0])]

In [ ]:
inputs_correct = tokenizer(query_correct, return_tensors="pt",padding=True,truncation=True).input_ids
inputs_wrong = tokenizer(query_wrong, return_tensors="pt",padding=True,truncation=True).input_ids

In [ ]:
outputs_correct = model_pt.generate(inputs_correct,max_new_tokens=max_target_length,num_beams=10,num_return_sequences=10)
outputs_wrong = model_pt.generate(inputs_wrong,max_new_tokens=max_target_length,num_beams=10,num_return_sequences=10)

In [ ]:
alternativas_corretas = tokenizer.batch_decode(outputs_correct, skip_special_tokens=True)
alternativas_erradas = tokenizer.batch_decode(outputs_wrong, skip_special_tokens=True)

In [ ]:
for x in alternativas_corretas:
  print(x)

banco de investimento e o Tribunal de Contas da União Europeia
Fundo Europeu de Investimento
banco Europeu de Investimento
Banco Europeu de Investimento
o financiamento de instituições não financeiras do Estado-membro da União Europeia
o financiamento de instituições não financeiras do Estado-Unite da União Europeia
banco, que representa uma autarquia da União Europeia
o financiamento de instituições não financeiras da União Europeia
o financiamento de projetos no setor de serviços financeiros do Banco Europeu
o financiamento de projetos no sector do metalúrgico


In [ ]:
for x in alternativas_erradas:
  print(x)

banco de investimento e o Tribunal de Contas da União Europeia
Fundo Europeu de Investimento
banco Europeu de Investimento
Banco Europeu de Investimento
o financiamento de instituições não financeiras do Estado-membro da União Europeia
o financiamento de instituições não financeiras do Estado-Unite da União Europeia
banco, que representa uma autarquia da União Europeia
o financiamento de instituições não financeiras da União Europeia
o financiamento de projetos no setor de serviços financeiros do Banco Europeu
o financiamento de projetos no sector do metalúrgico
